In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Load dataset
df = pd.read_csv("h2_cng_data.csv")

# Define input features and target variables
X = df[['H2_ratio', 'Pressure', 'Temperature']]
y = df[['Efficiency', 'Emissions']]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, y_train)

# Evaluate model
predictions = model.predict(X_test)
print("MSE:", mean_squared_error(y_test, predictions))

MSE: 3.2208820648951972


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Build model
nn_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(2)  # Two outputs: Efficiency and Emissions
])

nn_model.compile(optimizer='adam', loss='mean_squared_error')
nn_model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2)

2025-02-14 05:15:56.433276: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-14 05:15:56.450574: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-14 05:15:56.577527: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-14 05:15:56.581532: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-14 05:15:56.583034: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [7]:
!pip install deap==1.4.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.2/135.2 kB 5.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [9]:
from deap import base, creator, tools, algorithms
import random

# Define fitness function
def evaluate(individual):
    H2_ratio, pressure, temperature = individual
    predicted_efficiency, predicted_emissions = model.predict([[H2_ratio, pressure, temperature]])[0]
    return predicted_efficiency - predicted_emissions,  # Maximize efficiency, minimize emissions

# Set up GA
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, 0, 1)  # H2 ratio range
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=3)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=50)
algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, verbose=True)

/root/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/root/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/root/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/root/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/root/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/root/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not h

([[0.2298732223923568, 0.42400974563160576, 0.2884409593447267],
  [0.21991820425160225, 0.8220646127976683, 0.8417239753428973],
  [0.21991820425160225, 0.468545654572664, 0.6513747053695313],
  [0.4369616851659952, 0.8220646127976683, 0.2884409593447267],
  [0.21991820425160225, 0.3576030649639519, 0.6513747053695313],
  [0.35601220369303965, 0.8220646127976683, 0.6513747053695313],
  [0.9323329886234756, 0.42400974563160576, 0.7391264643297564],
  [0.9323329886234756, 0.3576030649639519, 0.2884409593447267],
  [0.9323329886234756, 0.3576030649639519, 0.2884409593447267],
  [0.21991820425160225, 0.8220646127976683, 0.6513747053695313],
  [0.2298732223923568, 0.29853392318473926, 0.6513747053695313],
  [0.21991820425160225, 0.42400974563160576, 0.6513747053695313],
  [0.9323329886234756, 0.8936012440459216, 0.2884409593447267],
  [0.4369616851659952, 0.8220646127976683, 0.075130029294558],
  [0.35601220369303965, 0.8220646127976683, 0.6513747053695313],
  [0.21991820425160225, 0.35760

In [11]:
import os
print(os.listdir())  # Lists all files in the current directory

['h2_cng_data.csv']


In [14]:
import os

file_path = "/path/to/h2_cng_model.pkl"  # Change this to the actual path

if os.path.exists(file_path):
    print("File found!")
else:
    print("File not found. Check the path and file location.")

File not found. Check the path and file location.


In [18]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Load dataset
df = pd.read_csv("h2_cng_data.csv")

# Define input features and target variables
X = df[['H2_ratio', 'Pressure', 'Temperature']]
y = df[['Efficiency', 'Emissions']]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, y_train)

# Save the model
model_path = "h2_cng_model.pkl"
with open(model_path, "wb") as file:
    pickle.dump(model, file)

print(f"Model saved at {model_path}")

Model saved at h2_cng_model.pkl


In [20]:
with open("h2_cng_model.pkl", "rb") as file:
    loaded_model = pickle.load(file)

In [22]:
pip install -U scikit-learn==1.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 39.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.1.3
    Uninstalling scikit-learn-1.1.3:
      Successfully uninstalled scikit-learn-1.1.3

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
import joblib
from sklearn.ensemble import RandomForestRegressor

# Train your model
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Save the model with the updated scikit-learn version
joblib.dump(model, "h2_cng_model.pkl")

['h2_cng_model.pkl']

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=540cdf78-a474-4530-b428-3245ce46c724' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>